In [301]:
from eth_utils import address
from web3 import Web3
import os
import solcx
from solcx import compile_standard, install_solc
from dotenv import load_dotenv
import json
import numpy as np
import random as rd
import hashlib 
from phe import paillier
import random as rd
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import rsa
from random import choice


In [302]:
listOfUID = [1,2,3,4]
listofBID = []
dictionaryOfPublicKeys = {}
dictionaryOfPrivateKeys = {}
dictOfRSAPubKey,dictOfRSAPriKey = {},{}
publicKeyAuc, privateKeyAuc,RSApublicKeyAuc, RSAprivateKeyAuc = 0,0,0,0

In [303]:
with open("./Storage.sol", "r") as file:
    simple_storage_file = file.read()

In [304]:
install_solc("0.8.2")
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"Storage.sol": {"content": simple_storage_file}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version="0.8.2",
)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)

In [305]:
# get bytecode
bytecode = compiled_sol["contracts"]["Storage.sol"]["Storage"]["evm"][
    "bytecode"
]["object"]

In [306]:
# get abi
abi = json.loads(
    compiled_sol["contracts"]["Storage.sol"]["Storage"]["metadata"]
)["output"]["abi"]

In [307]:
# set up connection
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:7545"))
chain_id = 1337
my_address = "0x19cfa72e6e0a5aF136111EAC2A96ed744b9b9627"
#private_key = os.getenv("PRIVATE_KEY")
private_key = "0xd54a7c95acc41adb4c5538dfcf8f5b7304f10ce94eb35d29dec7f801a99a74cd"

In [308]:
# initialize contract
Storage = w3.eth.contract(abi=abi, bytecode=bytecode)
nonce = w3.eth.get_transaction_count(my_address)
# set up transaction from constructor which executes when firstly
transaction = Storage.constructor().build_transaction(
    {"chainId": chain_id, "from": my_address, "nonce": nonce}
)
signed_tx = w3.eth.account.sign_transaction(transaction, private_key=private_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [309]:
# calling functions in contract block
# to work with a contract, you need abi and address

storage_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)
call_fun = storage_sol.functions.store2(42).build_transaction(
    {"chainId": chain_id, "from": my_address, "nonce": nonce + 1}
)
sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_key)
tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.rawTransaction)
tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)
# print(storage_sol.functions.store(45).transact())
print(storage_sol.functions.retrieve().call())
 

42


FROM this line the code for origiianl setup works


this phase consist of the registration phase

In [310]:
def bidderAndAuctioneerRegistration():
    global RSAprivateKeyAuc, RSApublicKeyAuc, privateKeyAuc,publicKeyAuc
    for i in range(len(listOfUID)):
        publicKeyAuc, privateKeyAuc = paillier.generate_paillier_keypair(n_length= 64)
        RSApublicKeyAuc, RSAprivateKeyAuc = rsa.newkeys(1048)
        listofBID.append(hashlib.sha256(str(listOfUID[i]).encode('utf-8')).hexdigest())
        x,y = paillier.generate_paillier_keypair(n_length=64)
        dictionaryOfPublicKeys[listofBID[i]],dictionaryOfPrivateKeys[listofBID[i]]  = x ,y
        (pubkey, privkey) = rsa.newkeys(1048)
        dictOfRSAPubKey[i],dictOfRSAPriKey[i] = pubkey, privkey 
    

        

In [311]:
bidderAndAuctioneerRegistration()

this phase consists of the bid submission phase

In [318]:
def bidSubmission():
    temp = rd.randint(0,len(listOfUID))
    bidder1 = listofBID[temp]
    temp = choice([i for i in range(0,len(listOfUID)) if i not in [temp]])
    bidder2 = listofBID[temp]
    b1rdnum = rd.randint(1,2000)
    b1rdbid = rd.randint(1,20)
    print(b1rdnum,b1rdbid)
    pb12 = rsa.encrypt(str((dictionaryOfPublicKeys[bidder2].raw_encrypt(b1rdbid + b1rdnum))).encode('utf8'), RSApublicKeyAuc)
    b2rdnum = rd.randint(1,2000)
    b2rdbid = rd.randint(1,20)
    print(b2rdnum,b2rdbid)
    pb21 = rsa.encrypt(str((dictionaryOfPublicKeys[bidder1].raw_encrypt(b2rdnum+ b2rdbid))).encode("utf8"), RSApublicKeyAuc)
    #call smartcontract scA frrom here to submit these values to the auctioneer
    pb12 = int(rsa.decrypt(pb12,RSAprivateKeyAuc).decode('utf8'))
    pb21 = int(rsa.decrypt(pb21,RSAprivateKeyAuc).decode('utf8'))
    #auctioneer adds its randim number frction here
    m1 = rd.randint(1,10000)
    print(m1)
    m2 = rd.randint(1,10000)
    pb12 = np.dot(pb12, dictionaryOfPublicKeys[bidder2].raw_encrypt(m1))
    pb21 = np.dot(pb21, dictionaryOfPublicKeys[bidder1].raw_encrypt(m1))
    # auctioneer forwards this info to b1 and b2 and they both add their random numbers
    pb21 = dictionaryOfPrivateKeys[bidder1].raw_decrypt(pb21)
    pb12 = dictionaryOfPrivateKeys[bidder2].raw_decrypt(pb12)
    print(pb21,pb12)

In [319]:
bidSubmission()

1070 13
1467 4
4349
5820 5432
